In [1]:
Nˣ, Nʸ, Nᶻ = 100, 100, 50

(100, 100, 50)

In [67]:
# δˣf2z, δʸf2z, and δᶻf2z calculate a difference in the x, y, and
# z-directions for a field defined at the cell faces/boundaries
# and projects it onto the cell centers.

# Input: Field defined at the u-faces, which has size (Nx+1, Ny, Nz).
# Output: Field defined at the cell centers, which has size (Nx, Ny, Nz).
function δˣf2z(f)
    Nx, Ny, Nz = size(f)[1] - 1, size(f)[2], size(f)[3]
    δf = zeros(Nx, Ny, Nz)
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[i+1, j, k] - f[i, j, k]
    end
    δf
end

# Input: Field defined at the v-faces, which has size (Nx, Ny+1, Nz).
# Output: Field defined at the cell centers, which has size (Nx, Ny, Nz).
function δʸf2z(f)
    Nx, Ny, Nz = size(f)[1], size(f)[2] - 1, size(f)[3]
    δf = zeros(Nx, Ny, Nz)
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[i, j+1, k] - f[i, j, k]
    end
    δf
end

# Input: Field defined at the v-faces, which has size (Nx, Ny, Nz+1).
# Output: Field defined at the cell centers, which has size (Nx, Ny, Nz).
function δᶻf2z(f)
    Nx, Ny, Nz = size(f)[1], size(f)[2], size(f)[3] - 1
    δf = zeros(Nx, Ny, Nz)
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[i, j, k+1] - f[i, j, k]
    end
    δf
end

δᶻf2z (generic function with 1 method)

In [74]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

# Input: Field defined at the cell centers, which has size (Nx, Ny, Nz).
# Output: Field defined at the u-faces, which has size (Nx+1, Ny, Nz).
function δˣz2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx+1, Ny, Nz)
    
    # Calculate δˣ in the interior.
    for k in 1:Nz, j in 1:Ny, i in 2:Nx
        δf[i, j, k] =  f[i, j, k] - f[i-1, j, k]
    end
    
    # Calculate δˣ at the left and right boundaries (the leftmost and rightmost faces are the
    # same in our periodic configuration).
    for k in 1:Nz, j in 1:Ny
        δ′ = f[1, j, k] - f[end, j, k]
        δf[1, j, k], δf[end, j, k] = δ′, δ′
    end
    
    δf
end

# Input: Field defined at the cell centers, which has size (Nx, Ny, Nz).
# Output: Field defined at the v-faces, which has size (Nx, Ny+1, Nz).
function δʸz2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny+1, Nz)
    
    # Calculate δʸ in the interior.
    for k in 1:Nz, j in 2:Ny, i in 1:Nx
        δf[i, j, k] =  f[i, j, k] - f[i, j-1, k]
    end
    
    # Calculate δʸ at the north and south boundaries (the leftmost and rightmost faces are the
    # same in our periodic configuration).
    for k in 1:Nz, i in 1:Nx
        δ′ = f[i, 1, k] - f[i, end, k]
        δf[i, 1, k], δf[i, end, k] = δ′, δ′
    end
    
    δf
end

# Input: Field defined at the cell centers, which has size (Nx, Ny, Nz).
# Output: Field defined at the w-faces, which has size (Nx, Ny, Nz+1).
function δᶻz2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz+1)
    
    # Calculate δᶻ in the interior.
    for k in 2:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[i, j, k] - f[i, j, k-1]
    end
    
    # Calculate δᶻ at the top and bottom boundaries (the leftmost and rightmost faces are the
    # same in our periodic configuration).
    for j in 1:Ny, i in 1:Nx
        δ′ = f[i, j, 1] - f[i, j, end]
        δf[i, j, 1], δf[i, j, end] = δ′, δ′
    end
    
    δf
end

δᶻz2f (generic function with 1 method)

In [92]:
# Input: Field defined at the cell centers, which has size (Nx, Ny, Nz).
# Output: Field defined at the u-faces, which has size (Nx+1, Ny, Nz).
function avgˣz2f(f)
    Nx, Ny, Nz = size(f)
    fa = zeros(Nx+1, Ny, Nz)
    
    # Calculate avgˣ in the interior.
    for k in 1:Nz, j in 1:Ny, i in 2:Nx
        fa[i, j, k] =  (f[i-1, j, k] + f[i, j, k]) / 2
    end
    
    # Calculate avgˣ at the left and right boundaries (the leftmost and rightmost faces are the
    # same in our periodic configuration).
    for k in 1:Nz, j in 1:Ny
        avg′ = (f[1, j, k] + f[end, j, k]) / 2
        fa[1, j, k], fa[end, j, k] = avg′, avg′
    end
    
    fa
end

# Input: Field defined at the cell centers, which has size (Nx, Ny, Nz).
# Output: Field defined at the v-faces, which has size (Nx, Ny+1, Nz).
function avgʸz2f(f)
    Nx, Ny, Nz = size(f)
    fa = zeros(Nx, Ny+1, Nz)
    
    # Calculate avgʸ in the interior.
    for k in 1:Nz, j in 2:Ny, i in 1:Nx
        fa[i, j, k] =  (f[i, j-1, k] + f[i, j, k]) / 2
    end
    
    # Calculate avgʸ at the north and south boundaries (the northmost and southtmost faces are the
    # same in our periodic configuration).
    for k in 1:Nz, i in 1:Nx
        avg′ = (f[i, 1, k] + f[i, end, k]) / 2
        fa[i, 1, k], fa[i, end, k] = avg′, avg′
    end
    
    fa
end

# Input: Field defined at the cell centers, which has size (Nx, Ny, Nz).
# Output: Field defined at the w-faces, which has size (Nx, Ny, Nz+1).
function avgᶻz2f(f)
    Nx, Ny, Nz = size(f)
    fa = zeros(Nx, Ny, Nz+1)
    
    # Calculate avgᶻ in the interior.
    for k in 2:Nz, j in 1:Ny, i in 1:Nx
        fa[i, j, k] =  (f[i, j, k-1] + f[i, j, k]) / 2
    end
    
    # Calculate avgᶻ at the top and bottom boundaries (the surface and bottom faces are the
    # same in our periodic configuration).
    for j in 1:Ny, i in 1:Nx
        avg′ = (f[i, j, 1] + f[i, j, end]) / 2
        fa[i, j, 1], fa[i, j, end] = avg′, avg′
    end
    
    fa
end

avgᶻz2f (generic function with 1 method)

In [100]:
# Input: fˣ is on u-face grid with size (Nx+1, Ny, Nz).
#        fʸ is on v-face grid with size (Nx, Ny+1, Nz).
#        fᶻ is on w-face grid with size (Nx, Ny, Nz+1).
# Output: ∇·̲f is on zone/cell center grid with size (Nx, Ny, Nz). 
function div_f2z(fˣ, fʸ, fᶻ)
    Vᵘ, Aˣ, Aʸ, Aᶻ = 1, 1, 1, 1
    (1/Vᵘ) * ( δˣf2z(Aˣ .* fˣ) + δʸf2z(Aʸ .* fʸ) + δᶻf2z(Aᶻ .* fᶻ) )
end

# Input: u is on u-face grid with size (Nx+1, Ny, Nz).
#        v is on v-face grid with size (Nx, Ny+1, Nz).
#        w is on w-face grid with size (Nx, Ny, Nz+1).
#        Q is on zone/cell center grid with size (Nx, Ny, Nz).
# Output: ∇·(u̲Q) is on zone/cell center grid with size (Nx, Ny, Nz). 
function div_flux_f2z(u, v, w, Q)
    Vᵘ, Aˣ, Aʸ, Aᶻ = 1, 1, 1, 1
    flux_x = Aˣ .* u .* avgˣz2f(Q)
    flux_y = Aʸ .* v .* avgʸz2f(Q)
    flux_z = Aᶻ .* w .* avgᶻz2f(Q)

    # Imposing zero vertical flux through the top and bottom layers.
    @. flux_z[:, :, 1] = 0
    @. flux_z[:, :, end] = 0

    (1/Vᵘ) .* (δˣf2z(flux_x) .+ δʸf2z(flux_y) .+ δᶻf2z(flux_z))
end

div_flux_f2z (generic function with 1 method)

In [103]:
κʰ = 1
κᵛ = 1
function laplacian_diffusion_z2z(Q)
    Vᵘ, Aˣ, Aʸ, Aᶻ = 1, 1, 1, 1
    
    κ∇Q_x = κʰ .* Aˣ .* δˣz2f(Q)
    κ∇Q_y = κʰ .* Aʸ .* δʸz2f(Q)
    κ∇Q_z = κᵛ .* Aᶻ .* δᶻz2f(Q)
    
    (1/Vᵘ) .* div_f2z(κ∇Q_x, κ∇Q_y, κ∇Q_z)
end

laplacian_diffusion_z2z (generic function with 1 method)

In [62]:
A = reshape([x^2 for x in 0:1330], (11, 11, 11));

In [27]:
δˣf2z(A);
δʸf2z(A);
δᶻf2z(A)

10×10×10 Array{Float64,3}:
[:, :, 1] =
 14641.0  17303.0  19965.0  22627.0  …  30613.0  33275.0  35937.0  38599.0
 14883.0  17545.0  20207.0  22869.0     30855.0  33517.0  36179.0  38841.0
 15125.0  17787.0  20449.0  23111.0     31097.0  33759.0  36421.0  39083.0
 15367.0  18029.0  20691.0  23353.0     31339.0  34001.0  36663.0  39325.0
 15609.0  18271.0  20933.0  23595.0     31581.0  34243.0  36905.0  39567.0
 15851.0  18513.0  21175.0  23837.0  …  31823.0  34485.0  37147.0  39809.0
 16093.0  18755.0  21417.0  24079.0     32065.0  34727.0  37389.0  40051.0
 16335.0  18997.0  21659.0  24321.0     32307.0  34969.0  37631.0  40293.0
 16577.0  19239.0  21901.0  24563.0     32549.0  35211.0  37873.0  40535.0
 16819.0  19481.0  22143.0  24805.0     32791.0  35453.0  38115.0  40777.0

[:, :, 2] =
 43923.0  46585.0  49247.0  51909.0  …  59895.0  62557.0  65219.0  67881.0
 44165.0  46827.0  49489.0  52151.0     60137.0  62799.0  65461.0  68123.0
 44407.0  47069.0  49731.0  52393.0     60379.0 

In [91]:
B = reshape([x^2 for x in 0:999], (10, 10, 10))

10×10×10 Array{Int64,3}:
[:, :, 1] =
  0  100  400   900  1600  2500  3600  4900  6400  8100
  1  121  441   961  1681  2601  3721  5041  6561  8281
  4  144  484  1024  1764  2704  3844  5184  6724  8464
  9  169  529  1089  1849  2809  3969  5329  6889  8649
 16  196  576  1156  1936  2916  4096  5476  7056  8836
 25  225  625  1225  2025  3025  4225  5625  7225  9025
 36  256  676  1296  2116  3136  4356  5776  7396  9216
 49  289  729  1369  2209  3249  4489  5929  7569  9409
 64  324  784  1444  2304  3364  4624  6084  7744  9604
 81  361  841  1521  2401  3481  4761  6241  7921  9801

[:, :, 2] =
 10000  12100  14400  16900  19600  22500  25600  28900  32400  36100
 10201  12321  14641  17161  19881  22801  25921  29241  32761  36481
 10404  12544  14884  17424  20164  23104  26244  29584  33124  36864
 10609  12769  15129  17689  20449  23409  26569  29929  33489  37249
 10816  12996  15376  17956  20736  23716  26896  30276  33856  37636
 11025  13225  15625  18225  21025  2402

In [95]:
@show size(δˣz2f(B))
@show size(δʸz2f(B))
@show size(δᶻz2f(B))
@show size(avgˣz2f(B))
@show size(avgʸz2f(B))
@show size(avgᶻz2f(B))
δˣz2f(B)
δʸz2f(B)
δᶻz2f(B)
avgˣz2f(B)
avgʸz2f(B)
avgᶻz2f(B)

size(δˣz2f(B)) = (11, 10, 10)
size(δʸz2f(B)) = (10, 11, 10)
size(δᶻz2f(B)) = (10, 10, 11)
size(avgˣz2f(B)) = (11, 10, 10)
size(avgʸz2f(B)) = (10, 11, 10)
size(avgᶻz2f(B)) = (10, 10, 11)


10×10×11 Array{Float64,3}:
[:, :, 1] =
 405000.0  414100.0  423400.0  432900.0  …  472900.0  483400.0  494100.0
 405901.0  415021.0  424341.0  433861.0     473941.0  484461.0  495181.0
 406804.0  415944.0  425284.0  434824.0     474984.0  485524.0  496264.0
 407709.0  416869.0  426229.0  435789.0     476029.0  486589.0  497349.0
 408616.0  417796.0  427176.0  436756.0     477076.0  487656.0  498436.0
 409525.0  418725.0  428125.0  437725.0  …  478125.0  488725.0  499525.0
 410436.0  419656.0  429076.0  438696.0     479176.0  489796.0  500616.0
 411349.0  420589.0  430029.0  439669.0     480229.0  490869.0  501709.0
 412264.0  421524.0  430984.0  440644.0     481284.0  491944.0  502804.0
 413181.0  422461.0  431941.0  441621.0     482341.0  493021.0  503901.0

[:, :, 2] =
 5000.0  6100.0  7400.0   8900.0  …  14600.0  16900.0  19400.0  22100.0
 5101.0  6221.0  7541.0   9061.0     14821.0  17141.0  19661.0  22381.0
 5204.0  6344.0  7684.0   9224.0     15044.0  17384.0  19924.0  22664.0
 5

In [104]:
Nx, Ny, Nz = 10, 10, 10
Ax = rand(Nx+1, Ny, Nz)
Ay = rand(Nx, Ny+1, Nz)
Az = rand(Nx, Ny, Nz+1)
AQ = rand(Nx, Ny, Nz)
@show size(Ax)
@show size(Ay)
@show size(Az)
@show size(AQ)
@show size(div_f2z(Ax, Ay, Az))
@show size(div_flux_f2z(Ax, Ay, Az, AQ))
@show size(laplacian_diffusion_z2z(AQ))
div_f2z(Ax, Ay, Az)
laplacian_diffusion_z2z(AQ)

size(Ax) = (11, 10, 10)
size(Ay) = (10, 11, 10)
size(Az) = (10, 10, 11)
size(AQ) = (10, 10, 10)
size(div_f2z(Ax, Ay, Az)) = (10, 10, 10)
size(div_flux_f2z(Ax, Ay, Az, AQ)) = (10, 10, 10)
size(laplacian_diffusion_z2z(AQ)) = (10, 10, 10)


10×10×10 Array{Float64,3}:
[:, :, 1] =
  0.812077  -0.748237   -1.10268   …   0.43041    0.756062   1.59229 
  1.57947    1.04534     1.84208      -1.67056   -1.09554    0.892026
  0.750357   0.787012   -0.609446     -0.283582   0.679666   0.804789
  2.50618   -1.86954    -2.25371      -2.5728     0.654327   1.11401 
  0.180463   2.16792    -1.08226      -0.526395  -3.40943   -0.341162
  0.433272  -0.692699   -1.21631   …   1.45903    0.253163   0.81027 
  1.38205   -0.0894214   1.70706      -1.72814   -1.22523    2.14678 
  1.05471    2.75007    -3.60505       1.78264   -2.06739    1.10713 
 -1.13164    2.10667    -2.12814       1.11344    3.34218   -2.31502 
 -1.88411   -1.46894     1.27075       0.986881  -2.83741    1.37979 

[:, :, 2] =
 -1.6261    -2.28056   -0.818941  …  -2.30575    -1.53501   -0.533297 
  2.0276    -0.220801   1.69518       1.41251     2.32525   -0.272771 
  1.50573    0.2416    -0.880919      2.93794    -2.18167   -0.261225 
 -4.72773    2.06045   -0.590969   